In [20]:
import pandas as pd
import numpy as np
import time
from twython import Twython, TwythonError, TwythonRateLimitError
import json 
import sklearn
from sklearn.feature_extraction import text
import pandas_profiling
from pandas_profiling.utils.cache import cache_file 

In [21]:

APP_KEY = 'SECRET CODE'
APP_SECRET = 'SECRET CODE'

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [22]:
#Use Twython to connect to twitter
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [103]:
#Search twitter for set handles and hashtags
query = '#gsmnp OR #GrandCanyon OR #rmnp OR #ZionNPS OR #Yosemite OR #NPS'

In [203]:
listed = pd.DataFrame()
request_pages = 450
for i in range(0, request_pages):
    if i == 0: #on intial request, search for query
        q_tweet = twitter.search(q = query, count = '100', lang = 'en')
    else: #for additional request, search query and use the previous minimum id value as starting max value
        q_tweet = twitter.search(q = query, count = '100', lang = 'en',
                                max_id = curr_since)
    t_df = pd.json_normalize(q_tweet['statuses']) #normalize data
    if len(t_df.index) == 0: #if the length of the dataframe is equal to 0, break
        break
    curr_since = min(t_df['id']) #initial minimum id
    curr_since = str(curr_since) #convert to string
    listed = listed.append(t_df) 
    time.sleep(2)

NameError: name 'referenced_tweets' is not defined

In [191]:
print(listed)

                        created_at                   id               id_str  \
0   Sat Nov 28 23:10:03 +0000 2020  1332824101736603660  1332824101736603660   
1   Sat Nov 28 23:10:01 +0000 2020  1332824094463680512  1332824094463680512   
2   Sat Nov 28 23:08:13 +0000 2020  1332823643643138054  1332823643643138054   
3   Sat Nov 28 22:52:06 +0000 2020  1332819584701894658  1332819584701894658   
4   Sat Nov 28 22:51:22 +0000 2020  1332819400618110978  1332819400618110978   
..                             ...                  ...                  ...   
0   Thu Nov 19 19:54:12 +0000 2020  1329513322958516226  1329513322958516226   
0   Thu Nov 19 19:54:12 +0000 2020  1329513322958516226  1329513322958516226   
0   Thu Nov 19 19:54:12 +0000 2020  1329513322958516226  1329513322958516226   
0   Thu Nov 19 19:54:12 +0000 2020  1329513322958516226  1329513322958516226   
1   Thu Nov 19 18:35:06 +0000 2020  1329493417227857922  1329493417227857922   

                                       

In [192]:
#Save the data to .csv
listed.to_csv('data_df.csv', index=False) #save test file so not continually pulling from Twitter

In [194]:
#Open file
df = pd.read_csv('data_df.csv', low_memory= False)
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Columns: 348 entries, created_at to quoted_status.place.bounding_box.coordinates
dtypes: bool(15), float64(99), int64(11), object(223)
memory usage: 5.6+ MB


In [195]:
print(df.head())

                       created_at                   id               id_str  \
0  Sat Nov 28 23:10:03 +0000 2020  1332824101736603660  1332824101736603660   
1  Sat Nov 28 23:10:01 +0000 2020  1332824094463680512  1332824094463680512   
2  Sat Nov 28 23:08:13 +0000 2020  1332823643643138054  1332823643643138054   
3  Sat Nov 28 22:52:06 +0000 2020  1332819584701894658  1332819584701894658   
4  Sat Nov 28 22:51:22 +0000 2020  1332819400618110978  1332819400618110978   

                                                text  truncated  \
0  Bucket List Alert!\nIn 3 days rafting trips to...       True   
1  Peaceful #Valley #totebag\nOn @society6! https...       True   
2  Today, #GSMNP. #alumcavetrail #greatsmokymount...       True   
3  RT @ImagesByJonEvan: Art for the Eyes! https:/...      False   
4  Is it time for you to renew your #nursing lice...       True   

                                              source  in_reply_to_status_id  \
0  <a href="https://www.hootsuite.com" rel=

In [196]:
print(df.columns)

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       ...
       'quoted_status.place.id', 'quoted_status.place.url',
       'quoted_status.place.place_type', 'quoted_status.place.name',
       'quoted_status.place.full_name', 'quoted_status.place.country_code',
       'quoted_status.place.country', 'quoted_status.place.contained_within',
       'quoted_status.place.bounding_box.type',
       'quoted_status.place.bounding_box.coordinates'],
      dtype='object', length=348)


In [197]:
#Check to see if any null values in text column
print(df['text'].isnull().values.any())

False


In [212]:
#Remove retweets from text column
df_nort = df[~df.text.str.contains('RT')]
print(df_nort['text'])

0       Bucket List Alert!\nIn 3 days rafting trips to...
1       Peaceful #Valley #totebag\nOn @society6! https...
2       Today, #GSMNP. #alumcavetrail #greatsmokymount...
4       Is it time for you to renew your #nursing lice...
6       Art for the Eyes! https://t.co/t05x0aBhJZ #yos...
                              ...                        
2195    Your NPS (net promoter score) is a fantastic t...
2196    Your NPS (net promoter score) is a fantastic t...
2197    Your NPS (net promoter score) is a fantastic t...
2198    Your NPS (net promoter score) is a fantastic t...
2199    We both loved 💚 💙 our vistit to the Grand Cany...
Name: text, Length: 1087, dtype: object


In [215]:
#Remove duplicates from text column
df_nort = df_nort.drop_duplicates(subset='text')
print(df_nort['text'])

0       Bucket List Alert!\nIn 3 days rafting trips to...
1       Peaceful #Valley #totebag\nOn @society6! https...
2       Today, #GSMNP. #alumcavetrail #greatsmokymount...
4       Is it time for you to renew your #nursing lice...
6       Art for the Eyes! https://t.co/t05x0aBhJZ #yos...
                              ...                        
2112    If you love #nature &amp; #animals and don't a...
2113    Yes, You Can Visit the Grand Canyon in Winter ...
2114    Hidden Forest offers a natural area among the ...
2116    Your NPS (net promoter score) is a fantastic t...
2199    We both loved 💚 💙 our vistit to the Grand Cany...
Name: text, Length: 995, dtype: object


In [216]:
#Separate tweet from other available data
tweet = df_nort['text']
print(tweet)

0       Bucket List Alert!\nIn 3 days rafting trips to...
1       Peaceful #Valley #totebag\nOn @society6! https...
2       Today, #GSMNP. #alumcavetrail #greatsmokymount...
4       Is it time for you to renew your #nursing lice...
6       Art for the Eyes! https://t.co/t05x0aBhJZ #yos...
                              ...                        
2112    If you love #nature &amp; #animals and don't a...
2113    Yes, You Can Visit the Grand Canyon in Winter ...
2114    Hidden Forest offers a natural area among the ...
2116    Your NPS (net promoter score) is a fantastic t...
2199    We both loved 💚 💙 our vistit to the Grand Cany...
Name: text, Length: 995, dtype: object


In [217]:
#Convert to lowercase letters for consistency 
tweet_lower = tweet.str.lower()
print(tweet_lower[0:60])

0      bucket list alert!\nin 3 days rafting trips to...
1      peaceful #valley #totebag\non @society6! https...
2      today, #gsmnp. #alumcavetrail #greatsmokymount...
4      is it time for you to renew your #nursing lice...
6      art for the eyes! https://t.co/t05x0abhjz #yos...
7      #sunset scenery photo of merced river (bottom)...
8      #hdr #sunset scenery photo of merced river sho...
10     steven and cary stayner: the tale of two broth...
11     wanna go rafting in the #grandcanyon? trips fo...
13     i am grateful for a beautiful earth and fun pe...
14     mimikyu goes the the #grandcanyon 💜\n#pokemon ...
18     just got checked into our room at tenaya lodge...
20     ✈️🎁do you love to travel? we got 15% savings a...
21     🤩we are always up for an adventure! 📷: @iankam...
22     and i think every service company should learn...
23     enjoying half-dome plein air, glacier point, y...
24     after almost three years living in arizona, i’...
27     ⛰ “grand canyon - travel

In [222]:
#Remove punctuation, remove \n
#Need to remove http, the following did not work:
#tweet_nopunc = tweet_lower.str.replace('http\SS+|www.\S+'', '', case=False) #Tried to remove http strings, but did not work
#tweet_nopunc = tweet_nopunc.str.replace('http+', ' ') #Same as above

tweet_nopunc = tweet_lower.str.replace('[^!#\w\s]','')
tweet_nopunc = tweet_nopunc.str.replace('\n',' ')

print(tweet_nopunc)

0       bucket list alert! in 3 days rafting trips to ...
1       peaceful #valley #totebag on society6! httpstc...
2       today #gsmnp #alumcavetrail #greatsmokymountai...
4       is it time for you to renew your #nursing lice...
6       art for the eyes! httpstcot05x0abhjz #yosemite...
                              ...                        
2112    if you love #nature amp #animals and dont alre...
2113    yes you can visit the grand canyon in winter p...
2114    hidden forest offers a natural area among the ...
2116    your nps net promoter score is a fantastic too...
2199    we both loved   our vistit to the grand canyon...
Name: text, Length: 995, dtype: object
